In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
from datetime import timedelta
import pandas as pd

def makeFig(df):
    processed_df = df.drop(columns=["Unnamed: 0", '상품명', '구매일(월)', 'ID', '거주지역', '물품대분류'])
    
    agg_dict = {'성별':'first', '결혼유무':'first', '연령':'first', '결제등록카드':'first', '유입경로':'first', '자녀여부':'first', '구매일':'first',
       '구매시월령(수정)':'first', '고객번호':'first', '구매금액': 'sum', '결제금액': 'sum', '배송시간':'first'
       }
    merged_data = processed_df.groupby('주문번호').agg(agg_dict).reset_index()
    merged_data['구매일'] = pd.to_datetime(merged_data['구매일']) 
    merged_data['구매횟수'] = merged_data.groupby(['고객번호', '구매일']).cumcount()
    merged_data['구매횟수'].fillna(0, inplace=True)
    merged_data['구매횟수'] = merged_data['구매횟수'].astype(int)
    merged_data = merged_data[merged_data['구매시월령(수정)'] < 29.5]

    rebuy_rate = merged_data.copy()

    rebuy_rate.sort_values(by='구매일', inplace=True, ascending=False)
    # Group by '고객번호' and calculate the difference between consecutive '구매일'
    rebuy_rate['구매일_diff'] = rebuy_rate.groupby('고객번호')['구매일'].diff()

    # Convert '구매일_diff' column to days
    rebuy_rate['구매일_diff'] = rebuy_rate['구매일_diff'] / timedelta(days=1)

    # Create a boolean mask to check if '구매일_diff' is less than or equal to 60 (2 months)
    rebuy_rate['재구매유무_1달이내'] = rebuy_rate['구매일_diff'] > - 32
    rebuy_rate['재구매유무_2달이내'] =  rebuy_rate['구매일_diff'] > - 61
    rebuy_rate['재구매유무_3달이내'] = rebuy_rate['구매일_diff'] > - 92


    rebuy_rate['재구매유무_1달이내'] = rebuy_rate['재구매유무_1달이내'].astype(int)
    rebuy_rate['재구매유무_2달이내'] = rebuy_rate['재구매유무_2달이내'].astype(int)
    rebuy_rate['재구매유무_3달이내'] = rebuy_rate['재구매유무_3달이내'].astype(int)
    rebuy_rate['구매연월'] = rebuy_rate['구매일'].dt.strftime("%Y%m")
    rebuy_rate.sort_values(by='구매일', inplace=True)
    # Convert boolean values to 1 and 0
    rebuy_rate_1 = rebuy_rate[['구매연월', '재구매유무_1달이내', '재구매유무_2달이내', '재구매유무_3달이내']]
    mean_repurchase_by_month = rebuy_rate_1.groupby('구매연월').mean()
    
    mrbm = mean_repurchase_by_month.reset_index()
    mrbm = mrbm[mrbm['구매연월'] <  '202008']
    mrbm.set_index(mrbm['구매연월'], inplace=True)

    fig = px.bar(mrbm, x=mrbm.index, y="재구매유무_1달이내", color=mrbm.index,
                color_discrete_sequence=["#56BDF4"], title='재구매유무_1달이내')
    fig.add_hline(y = 0.25, line_dash="dash", line_color="red")
    return fig



def make_page_repurchase(df):
    fig = makeFig(df)
    layout = [
        html.H1(children='재구매', style={'textAlign':'center'}),
        dcc.Graph(figure=fig, id='graph-repurchase')
    ]

        
    return layout

    

if __name__ == '__main__':
    import import_ipynb
    from jupyter_dash import JupyterDash
    from util.google import url_to_dataframe
    
    processed_df_link = "https://drive.google.com/file/d/1l9S5UU5f7L-OeKWhMRctCdSvVVxlFt_8/view?usp=sharing"

    ############################

    df = url_to_dataframe(processed_df_link)
    df['구매일'] = pd.to_datetime(df['구매일'])
    
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

    app = JupyterDash("6조 대쉬보드", external_stylesheets=external_stylesheets)
    
    app.layout = html.Div(make_page_repurchase(df))
    app.run_server(debug=True)